References:


1.   https://course19.fast.ai/videos/?lesson=12
2.   https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/?utm_source=blog&utm_medium=top-pretrained-models-nlp-article
3.   https://forums.fast.ai/t/how-to-load-pre-trained-model-wt103-from-local-directory/52585






In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

In [ ]:
!pip install fastai

In [1]:
#import libraries
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [2]:
#import dataset
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [3]:
#create dataframe
df = pd.DataFrame({'label': dataset.target, 'text': dataset.data})

In [4]:
df.shape

(11314, 2)

In [5]:
'''
binary classification problem
select labels 1 and 10 which correspond to ‘comp.graphics’ and ‘rec.sport.hockey’, respectively
'''
df = df[df['label'].isin([1,10])]
df = df.reset_index(drop=True)

In [6]:
#check evenness of data points 
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

PREPROCESS THE TEXT(we can do with spacy)

In [7]:
#tokenization
tokenized_doc = df['text'].apply(lambda x: x.split())

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
  t = ' '.join(tokenized_doc[i])
  detokenized_doc.append(t)

df['text'] = detokenized_doc

In [8]:
#split to train test sets
from sklearn.model_selection import train_test_split

#split data into training and validation sets
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [9]:
df_trn.shape, df_val.shape

((710, 2), (474, 2))

In [10]:
#Prepares language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

#Prepares classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab = data_lm.train_ds.vocab, bs=32)

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


PRETRAINING AND FINE TUNING

In [11]:
#get pretrained language model
learn = language_model_learner(data_lm, arch=AWD_LSTM, pretrained=True, drop_mult=0.7)

In [12]:
# train the learner object with learning rate = 1e-2, cyclic momentum https://arxiv.org/abs/1803.09820
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.587978,4.003945,0.279759,00:08


In [13]:
#save model to use for classification later
learn.save_encoder('ft_enc')

In [14]:
#use the data for classification
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.7)
#load previously saved model
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (710 items)
x: TextList
xxbos xxrep 8 . xxmaj it looks like the xxmaj edmonton xxmaj oilers just decided to take a xxmaj european xxunk this spring ... xxmaj ranford , xxmaj tugnutt , xxmaj benning , xxmaj manson , xxmaj smith , xxmaj buchberger , and xxmaj corson are playing for xxmaj canada . xxmaj podein and xxmaj weight are playing for the xxup us . xxmaj is xxmaj kravchuk playing for the xxmaj xxunk ... i know he had nagging injuries late in the season . xxmaj podein is an interesting case ... because he was eligible to play in xxmaj cape xxmaj breton in the xxup ahl playoffs like xxmaj kovalev , xxmaj zubov , and xxmaj andersson ... obviously xxmaj sather and xxmaj pocklington are not the total xxunk everyone makes them out to be ... certainly in this case they 've massively xxunk xxmaj paramount and the xxmaj new xxmaj york xxmaj rangers .,xxbos * xxmaj this is what xxunk me : xxrep 79 * xxmaj speaking of " die hard " , that '

In [16]:
#fitting again
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.330860,0.244025,0.928270,00:20


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [17]:
#get the predictions from validation set and show in crosstab to check correctness
preds, targets = learn.get_preds()
predictions = np.argmax(preds, axis=1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,225,25
1,9,215
